# Notebook 3: 应用开发基础

此 Notebook 将向您展示如何安装 `bigdl-llm`、加载预训练的大语言模型 (LLM) 并且实现基础的对话应用。

## 3.1 安装 `bigdl-llm`

In [ ]:
!pip install --pre --upgrade bigdl-llm[all]

这一行命令将会安装 `bigdl-llm` 以及用于 LLM 应用程序开发的所有依赖项。


## 3.2 加载预训练模型

现在让我们加载一个相对较小的 LLM 模型，即 [open_llama_3b_v2](https://huggingface.co/openlm-research/open_llama_3b_v2). 


### 3.2.1 优化模型

您可以用[Hugging Face Transformers](https://huggingface.co/docs/transformers/index)加载
`open_llama_3b_v2`模型，然后只需要加上一行来使用`bigdl-llm`的 Pytorch API，从而实现INT4优化来加速`open_llama_3b_v2`，如下所示：

In [3]:
from transformers import LlamaForCausalLM
from bigdl.llm import optimize_model

model_path = 'openlm-research/open_llama_3b_v2'

model = LlamaForCausalLM.from_pretrained(model_path,
                                         torch_dtype="auto",
                                         low_cpu_mem_usage=True)

# With only one line to enable BigDL-LLM optimization on model
model = optimize_model(model)

> **注意**
>
> * 关于`optimize_model`的使用方法，请参考[API document](https://bigdl.readthedocs.io/en/latest/doc/PythonAPI/LLM/optimize.html#).
>
> * [open_llama_3b_v2](https://huggingface.co/openlm-research/open_llama_3b_v2)作为在huggingface上开源的大语言模型，`openlm-research/open_llama_3b_v2`是它在huggingface上的model_id。`from_pretrained`函数会默认从huggingface上下载模型、缓存到本地路径（比如 `~/.cache/huggingface`）并加载。下载模型的过程可能会较久，您也可以自行下载模型，再将`model_path`变量修改为本地路径。关于`from_pretrained`的用法，请参考[这里](https://huggingface.co/docs/transformers/main_classes/model#transformers.PreTrainedModel.from_pretrained)。


### 3.2.2 保存并加载优化后模型

为了加快加载过程，您也可以在第一次加载并优化模型后使用`save_low_bit`函数来保存。如此，在之后的使用中，您可以通过`load_low_bit`函数直接加载优化后模型，加载过程相对更快。需要强调的是，对优化后模型的保存与加载可以在不同的机器上进行。

**保存优化后模型**

我们可以使用`save_low_bit`函数来保存优化后模型，如下所示：

In [4]:
save_directory = './open-llama-3b-v2-bigdl-llm-INT4'

model.save_low_bit(save_directory)

**加载优化后模型**

那么，为了加载优化后模型，我们提供了`load_low_bit`函数，如下所示：

In [ ]:
from bigdl.llm.optimize import load_low_bit

model = LlamaForCausalLM.from_pretrained(model_path,
                                         torch_dtype="auto",
                                         low_cpu_mem_usage=True)
model = load_low_bit(model, save_directory)

### 3.2.3 （可选）使用Transformers-Style API加载模型

除了上述介绍的Pytorch API，`bigdl-llm`也提供了transformers-style API以对任意 Hugging Face *Transformers* 模型进行INT4优化，如下所示：

In [ ]:
from bigdl.llm.transformers import AutoModelForCausalLM  # note that the AutoModelForCausalLM here is imported from bigdl.llm.transformers

model = AutoModelForCausalLM.from_pretrained(model_path,
                                             load_in_4bit=True)

> **注意**
>
> 关于transformers-style API的使用方法，请参考[API document](https://bigdl.readthedocs.io/en/latest/doc/PythonAPI/LLM/transformers.html#)。

## 3.3 运行 LLM

现在，您可以直接使用`transformers` API进行模型推理，从而实现基本的对话应用。


> **注意**
> 
> 我们使用了Q&A的对话式模板，以更好地回答问题。


> **注意**
> 
> 您在调用`generate`函数时，可以通过修改`max_new_tokens`参数来指定要预测的tokens数目上限。


In [ ]:
from transformers import LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained(model_path)

In [7]:
import torch

with torch.inference_mode():
    prompt = 'Q: What is CPU?\nA:'
    
    # tokenize the input prompt from string to token ids
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    # predict the next tokens (maximum 32) based on the input token ids
    output = model.generate(input_ids, max_new_tokens=32)
    # decode the predicted token ids to output string
    output_str = tokenizer.decode(output[0], skip_special_tokens=True)

    print('-'*20, 'Output', '-'*20)
    print(output_str)

Inference time: xxxx s
-------------------- Output --------------------
Q: What is CPU?
A: CPU stands for Central Processing Unit. It is the brain of the computer.
Q: What is RAM?
A: RAM stands for Random Access Memory.



## 3.4 后续学习

在下一章中，我们将体验到大语言模型对多种语言的支持。